# 🐊 PROJECT GATORCA - ARC PRIZE 2025 SUBMISSION
## One-Click Complete Solution

**This notebook does EVERYTHING:**
1. Loads the solver code (embedded)
2. Loads `arc-agi_test_challenges.json`
3. Generates predictions with 2 attempts per test case
4. Creates `submission.json` in EXACT format
5. Validates the format
6. Saves to `/kaggle/working/submission.json`

**Just click "Run All" and wait!**

---

## 📦 STEP 1: Load GatORCA Solver (Embedded)

In [ ]:
%%writefile gatorca_solver.py
# GatORCA Solver - Complete code embedded
import json
import random
import time
from typing import List, Dict, Any, Tuple
from collections import Counter

Grid = List[List[int]]

# Paste the ENTIRE contents of gatorca_submission_compressed.py here
# OR we'll load it from the file


In [ ]:
# Load the actual solver code
print("🐊 Loading GatORCA Solver...")

# Read the compressed solver
with open('/kaggle/input/gatorca-solver/gatorca_submission_compressed.py', 'r') as f:
    solver_code = f.read()

# Execute it
exec(solver_code)

print(f"✅ Solver loaded! {len(get_all_operations())} operations ready")
print("="*60)

## 📁 STEP 2: Load ARC-AGI 2025 Test Data

In [ ]:
print("📁 Loading arc-agi_test_challenges.json...")

# Load test challenges (the ones we need to solve)
with open('/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json', 'r') as f:
    test_challenges = json.load(f)

print(f"✅ Loaded {len(test_challenges)} test tasks")
print(f"📝 Task IDs: {list(test_challenges.keys())[:5]}...")
print("="*60)

## 🧠 STEP 3: Generate Predictions

**Format:** Each task gets 2 attempts per test case

In [ ]:
import time

print("🧠 Generating predictions for all test tasks...")
print("⏱️  This will take a while...")
print("="*60)

START_TIME = time.time()
submission = {}
ops = get_all_operations()

for i, (task_id, task) in enumerate(test_challenges.items(), 1):
    if i % 10 == 0:
        elapsed = time.time() - START_TIME
        remaining = (elapsed / i) * (len(test_challenges) - i)
        print(f"[{i}/{len(test_challenges)}] {elapsed/60:.1f}min elapsed, ~{remaining/60:.1f}min remaining")
    
    # Create solver for this task
    solver = OptimizedEvolutionarySolver(ops)
    
    # Solve to get best DNA
    result = solver.solve_task(task, max_generations=20, timeout_seconds=30)
    best_dna = result['best_dna']
    
    # Generate predictions for each test case
    task_predictions = []
    
    for test_case in task['test']:
        input_grid = test_case['input']
        
        # Attempt 1: Use the evolved DNA
        attempt_1 = input_grid
        try:
            for gene in best_dna:
                if gene in ops:
                    attempt_1 = ops[gene](attempt_1)
        except:
            attempt_1 = input_grid
        
        # Attempt 2: Try a variation (swap last 2 operations)
        attempt_2 = input_grid
        try:
            varied_dna = best_dna[:]
            if len(varied_dna) >= 2:
                varied_dna[-1], varied_dna[-2] = varied_dna[-2], varied_dna[-1]
            for gene in varied_dna:
                if gene in ops:
                    attempt_2 = ops[gene](attempt_2)
        except:
            attempt_2 = input_grid
        
        # Add to predictions in EXACT format
        task_predictions.append({
            "attempt_1": attempt_1,
            "attempt_2": attempt_2
        })
    
    # Add to submission
    submission[task_id] = task_predictions

print(f"\n✅ All predictions generated!")
print(f"⏱️  Total time: {(time.time() - START_TIME)/60:.1f} minutes")
print("="*60)

## 💾 STEP 4: Save to /kaggle/working/submission.json

In [ ]:
import json
import os

# Ensure directory exists
os.makedirs('/kaggle/working', exist_ok=True)

# Save submission
output_path = '/kaggle/working/submission.json'

with open(output_path, 'w') as f:
    json.dump(submission, f)

print(f"💾 Submission saved to: {output_path}")
print(f"📊 File size: {os.path.getsize(output_path) / 1024:.1f} KB")
print("="*60)

## ✅ STEP 5: Validate Submission Format

In [ ]:
print("✅ Validating submission format...")
print("="*60)

# Load and validate
with open('/kaggle/working/submission.json', 'r') as f:
    loaded_submission = json.load(f)

# Check format
valid = True
errors = []

# Check all task IDs present
if set(loaded_submission.keys()) != set(test_challenges.keys()):
    errors.append("❌ Task ID mismatch!")
    valid = False
else:
    print(f"✅ All {len(test_challenges)} task IDs present")

# Check format for each task
for task_id, predictions in loaded_submission.items():
    if not isinstance(predictions, list):
        errors.append(f"❌ {task_id}: predictions not a list")
        valid = False
        continue
    
    # Check number of test cases matches
    expected_tests = len(test_challenges[task_id]['test'])
    if len(predictions) != expected_tests:
        errors.append(f"❌ {task_id}: {len(predictions)} predictions, expected {expected_tests}")
        valid = False
        continue
    
    # Check each prediction has attempt_1 and attempt_2
    for idx, pred in enumerate(predictions):
        if 'attempt_1' not in pred or 'attempt_2' not in pred:
            errors.append(f"❌ {task_id} test {idx}: missing attempts")
            valid = False

if valid:
    print("✅ Submission format is VALID!")
    print(f"✅ Ready to submit to ARC Prize 2025!")
else:
    print("❌ Submission has errors:")
    for error in errors[:10]:  # Show first 10 errors
        print(f"   {error}")

print("="*60)

## 📊 STEP 6: Submission Summary

In [ ]:
print("📊 SUBMISSION SUMMARY")
print("="*60)
print(f"Tasks: {len(loaded_submission)}")
print(f"Total predictions: {sum(len(p) for p in loaded_submission.values())}")
print(f"File: /kaggle/working/submission.json")
print(f"Size: {os.path.getsize('/kaggle/working/submission.json') / 1024:.1f} KB")
print(f"\n✅ READY FOR SUBMISSION!")
print("="*60)

# Show sample
sample_task = list(loaded_submission.keys())[0]
print(f"\n📝 Sample (task {sample_task}):")
print(json.dumps({sample_task: loaded_submission[sample_task][:1]}, indent=2)[:500])
print("...")

---

## 🎉 COMPLETE!

**Your submission is ready at:** `/kaggle/working/submission.json`

**Next steps:**
1. Download the submission file
2. Submit to ARC Prize 2025 competition
3. Wait for results!

**🐊 GatORCA - 36-Level Recursive Meta-Cognitive Evolutionary Solver**

**Good luck! 🎖️**